<a href="https://colab.research.google.com/github/Hannah250/PROJECTS/blob/main/Machine_Learning_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
df = pd.read_csv('https://query.data.world/s/wh6j7rxy2hvrn4ml75ci62apk5hgae')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df.head()

In [ ]:
df.isna().sum()

country               0
year                  0
country_code          0
record                0
crop_land         20472
grazing_land      20472
forest_land       20472
fishing_ground    20473
built_up_land     20473
carbon            20473
total                 9
QScore                1
dtype: int64

In [ ]:
df=df.dropna()
df.isna().sum()

country           0
year              0
country_code      0
record            0
crop_land         0
grazing_land      0
forest_land       0
fishing_ground    0
built_up_land     0
carbon            0
total             0
QScore            0
dtype: int64

In [ ]:
df[ 'QScore' ] = df[ 'QScore' ].replace([ '1A' ], '2A' ) 
df.QScore.value_counts()

3A    51473
2A      240
Name: QScore, dtype: int64

In [ ]:
df_2A = df[df.QScore== '2A' ] 
df_3A = df[df.QScore== '3A' ].sample( 350 ) 
data_df = df_2A.append(df_3A)

In [ ]:
import sklearn.utils
data_df = sklearn.utils.shuffle(data_df)
data_df = data_df.reset_index(drop=True)
data_df.shape
data_df.QScore.value_counts()

3A    350
2A    240
Name: QScore, dtype: int64

In [ ]:
data_df.dtypes

country            object
year                int64
country_code        int64
record             object
crop_land         float64
grazing_land      float64
forest_land        object
fishing_ground    float64
built_up_land     float64
carbon            float64
total             float64
QScore             object
dtype: object

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
data_df.record = encoder.fit_transform(data_df.record)

In [ ]:
data_df.record

0      5
1      6
2      1
3      0
4      2
      ..
585    0
586    5
587    5
588    4
589    5
Name: record, Length: 590, dtype: int64

In [ ]:
data_df = data_df.drop(columns=['country_code', 'country', 'year'])
X = data_df.drop(columns = 'QScore')
y = data_df['QScore']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
y_train.value_counts()

3A    245
2A    168
Name: QScore, dtype: int64

In [ ]:
x_train.record

285    1
113    5
18     3
76     6
206    6
      ..
277    0
9      7
359    7
192    2
559    3
Name: record, Length: 413, dtype: int64

In [ ]:
import imblearn
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=1)
x_train_balanced, y_balanced = smote.fit_resample(x_train, y_train)
y_train.value_counts()

3A    245
2A    168
Name: QScore, dtype: int64

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalised_train_df = scaler.fit_transform(x_train_balanced.drop(columns=['record']))
normalised_train_df = pd.DataFrame(normalised_train_df, columns=x_train_balanced.drop(columns=['record']).columns)
normalised_train_df['record'] = x_train_balanced['record']

In [ ]:
x_test = x_test.reset_index(drop=True)
normalised_test_df = scaler.transform(x_test.drop(columns=['record']))
normalised_test_df = pd.DataFrame(normalised_test_df, columns=x_test.drop(columns=['record']).columns)
normalised_test_df['record'] = x_test['record']

In [ ]:
##Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(normalised_train_df, y_balanced)
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,                   intercept_scaling=1, l1_ratio=None, max_iter=100,                   multi_class='auto', n_jobs=None, penalty='l2',                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,                   warm_start=False)

LogisticRegression()

In [ ]:
##Measuring Classification performance

In [ ]:
#cross validation score

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(log_reg, normalised_train_df, y_balanced, cv=5,                     
   scoring='f1_macro')
scores

array([0.46344605, 0.51021295, 0.34693878, 0.51      , 0.46344605])

In [ ]:
#confusion matrix

In [ ]:
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix
new_predictions = log_reg.predict(normalised_test_df)
cnf_mat = confusion_matrix(y_true=y_test, y_pred=new_predictions, labels=['2A', '3A'])
cnf_mat

array([[63,  9],
       [76, 29]])

In [ ]:
##Metrics

In [ ]:
#Accuracy

In [ ]:
accuracy = accuracy_score(y_true = y_test, y_pred = new_predictions)
print('Accuracy: {}'.format(round(accuracy*100, 2)))

Accuracy: 51.98


In [ ]:
#Precision

In [ ]:
precision = precision_score(y_true=y_test, y_pred=new_predictions, pos_label='2A')
print('Presicion: {}'.format(round(precision*100), 2))

Presicion: 45


In [ ]:
#Recall

In [ ]:
recall = recall_score(y_true=y_test, y_pred=new_predictions, pos_label='2A')
print('Recall: {}'.format(round(recall*100), 2))

Recall: 88


In [ ]:
#F1_score

In [ ]:
f1 = f1_score(y_true=y_test, y_pred=new_predictions, pos_label='2A')
print('F1: {}'.format(round(f1*100), 2))

F1: 60


In [ ]:
#K Fold

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
kf.split(normalised_train_df)
f1_scores = []

In [ ]:
for train_index, test_index in kf.split(normalised_train_df):  
  x_train, x_test = normalised_train_df.iloc[train_index], normalised_train_df.iloc[test_index]  
  y_train, y_test = y_balanced[train_index], y_balanced[test_index] 
  model = LogisticRegression().fit(x_train, y_train)   
  f1_scores.append(f1_score(y_true = y_test, y_pred = model.predict(x_test),pos_label = '2A')*100)
f1_scores

[52.38095238095239,
 55.38461538461539,
 61.29032258064516,
 59.01639344262294,
 0.0]

In [ ]:
#Stratified K-Fold

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
f1_scores = []
for train_index, test_index in skf.split(normalised_train_df, y_balanced):   
  x_train, x_test = np.array(normalised_train_df)[train_index], np.array(normalised_train_df)[test_index]  
  y_train, y_test = y_balanced[train_index], y_balanced[test_index]  
  model = LogisticRegression().fit(x_train, y_train)     
  f1_scores.append(f1_score(y_true = y_test, y_pred = model.predict(x_test),pos_label = '2A')*100)
f1_scores

[54.20560747663551,
 70.22900763358778,
 56.91056910569105,
 48.54368932038835,
 65.07936507936508]

In [ ]:
#LeaveOneOut

In [ ]:
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
scores = cross_val_score(LogisticRegression(), normalised_train_df, y_balanced, cv=loo, scoring='f1_macro')
average_score = scores.mean() * 100
average_score

51.22448979591837

In [ ]:
#Tree Based Methods and SVM

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dec_tree = DecisionTreeClassifier()
dec_tree.fit(normalised_train_df, y_balanced)

DecisionTreeClassifier()